In [13]:
import pandas
from scipy import stats
from scipy.stats import pearsonr

In [2]:
indp = pandas.read_csv("industries_short_names.csv")
rac1p = pandas.read_csv("RAC1P.csv")
cow = pandas.read_csv("COW.csv", delimiter=':')


In [3]:
indp_dict = dict(zip(indp.Code, indp.Name))
rac1p_dict = dict(zip(rac1p.Id, rac1p.Race))
cow_dict = dict(zip(cow.b, cow['Not Applicable']))

gender_dict = dict()
gender_dict[1] = 'Male'
gender_dict[2] = 'Female'

In [4]:
features = []
features.extend(list(set(indp_dict.values())))
features.extend(rac1p_dict.values())
features.extend(cow_dict.values())
features.extend(gender_dict.values())
print(len(features))


41


In [5]:
features

['EDUCATION',
 'ADMINISTRATION',
 'EXTRACTION',
 'TRANSPORT',
 'CONSTRUCTION',
 'AGRICULTURE',
 'UNEMPLOYED',
 'FINANCE',
 'INFORMATION_TECH',
 'RETAIL',
 'COMMUNITY_SERVICES',
 'MEDICAL',
 'UTILITIES',
 'WHOLESALE',
 'MANUFACTURING',
 'MILITARY',
 'DOMESTIC_SERVICES',
 'ENTERTAINMENT',
 'SERVICES',
 'White',
 'Black or African American',
 'American Indian',
 'Alaska Native',
 'American Indian and Alaska Native tribes specified',
 'Asian',
 'Native Hawaiian',
 'Some Other Race',
 'Two or More Races',
 'Private charitable company employee',
 'Not Applicable',
 'Not Applicable',
 'Private profit company employee',
 'Self-employed not own business',
 'Local government employee',
 'State government employee',
 'Non paying family business',
 'Federal government employee',
 'Unemployed',
 'Self-employed own business',
 'Male',
 'Female']

In [6]:
import numpy
#chunk = pandas.read_csv("reduced_merged_data\\2013_1_reduced_merged.csv")

In [7]:
i = 0
for chunk in pandas.read_csv("../big_data/reduced_merged/2002_1_reduced_merged.csv", chunksize = 10000):
    chunk = chunk.dropna(subset=['INDUSTRY','CATEGORY_OF_WORK'])
    chunk = chunk.reset_index()
    print("Working on chunk: ",i, " with number of rows: ",len(chunk))
    extra_feature_cols = pandas.DataFrame(0, index=numpy.arange(len(chunk)), columns=features)
    chunk = pandas.concat([chunk, extra_feature_cols], axis=1)

    for index, row in chunk.iterrows():
        if row.INDUSTRY == 'INDUSTRY':
            continue
        try:
            chunk.loc[index, cow_dict[str(int(row['CATEGORY_OF_WORK']))]] = 1
        except:
            pass

        try:
            chunk.loc[index, gender_dict[int(row['GENDER'])]] = 1
        except:
            pass
        try:
            chunk.loc[index, rac1p_dict[row['RACE']]] = 1
        except:
            pass

        if row.HISPANIC == 1:
            chunk.loc[index, 'HISPANIC'] = 0
        else:
            chunk.loc[index, 'HISPANIC'] = 1
        try:
            chunk.loc[index, indp_dict[row['INDUSTRY']]] = 1
        except:
            pass
    
    print("Writing chunk: ",i)
    if i == 0:
        chunk.to_csv("./flattened_2002.csv", index=False)
    else:
        chunk.to_csv("./flattened_2002.csv", index=False, mode = 'a', header=False)
    i += 1

Working on chunk:  0  with number of rows:  5829
Writing chunk:  0
Working on chunk:  1  with number of rows:  6135
Writing chunk:  1
Working on chunk:  2  with number of rows:  6014
Writing chunk:  2
Working on chunk:  3  with number of rows:  5687
Writing chunk:  3
Working on chunk:  4  with number of rows:  5848
Writing chunk:  4
Working on chunk:  5  with number of rows:  5950
Writing chunk:  5
Working on chunk:  6  with number of rows:  5919
Writing chunk:  6
Working on chunk:  7  with number of rows:  5912
Writing chunk:  7
Working on chunk:  8  with number of rows:  5925
Writing chunk:  8
Working on chunk:  9  with number of rows:  5979
Writing chunk:  9
Working on chunk:  10  with number of rows:  5892
Writing chunk:  10
Working on chunk:  11  with number of rows:  5890
Writing chunk:  11
Working on chunk:  12  with number of rows:  5913
Writing chunk:  12
Working on chunk:  13  with number of rows:  5948
Writing chunk:  13
Working on chunk:  14  with number of rows:  5919
Writ

In [40]:
flattened = pandas.read_csv("./flattened_20")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:

flattened = flattened[['HISPANIC','EDUCATION', 'income', 'AGE','RETAIL','COMMUNITY_SERVICES', 'FINANCE', 'WHOLESALE','DOMESTIC_SERVICES', 'ENTERTAINMENT', 'ADMINISTRATION', 'UTILITIES','EXTRACTION', 'INFORMATION_TECH', 'CONSTRUCTION', 'SERVICES','MILITARY', 'UNEMPLOYED', 'MANUFACTURING', 'AGRICULTURE', 'MEDICAL','TRANSPORT', 'White', 'Black or African American', 'American Indian','Alaska Native', 'American Indian and Alaska Native tribes specified','Asian', 'Native Hawaiian', 'Some Other Race', 'Two or More Races', 'Private profit company employee','Local government employee', 'Private charitable company employee','Federal government employee', 'State government employee','Self-employed own business', 'Self-employed not own business','Unemployed', 'Non paying family business', 'Male', 'Female']]
flattened = flattened[flattened.AGE != 'AGE']

flattened.EDUCATION = flattened.EDUCATION.astype(float)
flattened.EDUCATION = stats.zscore(flattened.EDUCATION)

flattened.AGE = flattened.AGE.astype(int)
flattened.AGE = stats.zscore(flattened.AGE)

flattened.income = flattened.income.astype(float)
flattened.income = stats.zscore(flattened.income)

#df_for_regression = df_for_regression[df_for_regression.D != 'GENDER']
len(flattened)

651252

In [53]:
df_for_regression_y = flattened[['income']]
df_for_regression_x = flattened[['HISPANIC','EDUCATION', 'AGE','RETAIL','COMMUNITY_SERVICES', 'FINANCE', 'WHOLESALE','DOMESTIC_SERVICES', 'ENTERTAINMENT', 'ADMINISTRATION', 'UTILITIES','EXTRACTION', 'INFORMATION_TECH', 'CONSTRUCTION', 'SERVICES','MILITARY', 'UNEMPLOYED', 'MANUFACTURING', 'AGRICULTURE', 'MEDICAL','TRANSPORT', 'White', 'Black or African American', 'American Indian','Alaska Native', 'American Indian and Alaska Native tribes specified','Asian', 'Native Hawaiian', 'Some Other Race', 'Two or More Races', 'Private profit company employee','Local government employee', 'Private charitable company employee','Federal government employee', 'State government employee','Self-employed own business', 'Self-employed not own business','Unemployed', 'Non paying family business', 'Male', 'Female']]
column_list = df_for_regression_x.columns

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

regr = linear_model.Ridge()
regr.fit(df_for_regression_x, df_for_regression_y)

coeffs = regr.coef_[0]

column_to_coeff = dict()
for i in range(len(coeffs)):
    column_to_coeff[column_list[i]] = coeffs[i]
    
print(column_to_coeff)

{'ADMINISTRATION': -0.71241017298674403, 'EXTRACTION': -0.41245723997607397, 'TRANSPORT': -0.65909929625256325, 'Self-employed not own business': -0.057546739275223281, 'Self-employed own business': 0.58681790053152427, 'Non paying family business': -0.35314350483444584, 'Private profit company employee': -0.076810450467902316, 'UNEMPLOYED': -0.68902810183383234, 'Female': -0.20537758931392222, 'AGRICULTURE': -0.86273680811377573, 'White': 0.72338506069044273, 'HISPANIC': -0.024809179173996793, 'CONSTRUCTION': -0.71429798645855713, 'State government employee': 0.08735557285366638, 'MANUFACTURING': -0.53105609413054566, 'FINANCE': -0.34195399376814389, 'Asian': 0.70815359839415548, 'AGE': 0.15801759898223372, 'Alaska Native': 0.67783010272836031, 'EDUCATION': 0.33649318927660687, 'Native Hawaiian': 0.64567770437937977, 'RETAIL': -0.79215175863566478, 'Local government employee': 0.072884073908704453, 'Some Other Race': 0.70405338016234131, 'American Indian and Alaska Native tribes speci

In [52]:
pearsonr(flattened['Federal government employee'], flattened['Unemployed'])
flattened['White'].astype(int).corr( flattened['income'])

0.080330665471772617

In [79]:
dataset = pandas.read_csv("./flattened_2002.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [61]:
len(dataset.columns)

52

In [67]:
features = list(dataset.columns)
print(list(features))
index = features.index("income")
features = features[index:] + features[:index]
features
dataset = dataset[features]
#index

['index', 'SERIALNO', 'RACE', 'HISPANIC', 'INDUSTRY', 'CATEGORY_OF_WORK', 'EDUCATION', 'income', 'AGE', 'GENDER', 'DATA_FROM_YEAR', 'EDUCATION.1', 'ADMINISTRATION', 'EXTRACTION', 'TRANSPORT', 'CONSTRUCTION', 'AGRICULTURE', 'UNEMPLOYED', 'FINANCE', 'INFORMATION_TECH', 'RETAIL', 'COMMUNITY_SERVICES', 'MEDICAL', 'UTILITIES', 'WHOLESALE', 'MANUFACTURING', 'MILITARY', 'DOMESTIC_SERVICES', 'ENTERTAINMENT', 'SERVICES', 'White', 'Black or African American', 'American Indian', 'Alaska Native', 'American Indian and Alaska Native tribes specified', 'Asian', 'Native Hawaiian', 'Some Other Race', 'Two or More Races', 'Private charitable company employee', 'Not Applicable', 'Not Applicable.1', 'Private profit company employee', 'Self-employed not own business', 'Local government employee', 'State government employee', 'Non paying family business', 'Federal government employee', 'Unemployed', 'Self-employed own business', 'Male', 'Female']


In [70]:
dataset.head(1000).to_csv("./flattened_2002_1000.csv")

In [4]:
dataset = pandas.read_csv("./flattened_2002.csv")

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,47,48,49,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [18]:
year = 2016
#while(year < 2016):
dataset = pandas.read_csv("./flattened_files/flattened_files/flattened_" + str(year) + ".csv")
dataset = dataset[['income', 'HISPANIC', 'DATA_FROM_YEAR', 'EDUCATION', 'AGE','RETAIL','COMMUNITY_SERVICES', 'FINANCE', 'WHOLESALE','DOMESTIC_SERVICES', 'ENTERTAINMENT', 'ADMINISTRATION', 'UTILITIES','EXTRACTION', 'INFORMATION_TECH', 'CONSTRUCTION', 'SERVICES','MILITARY', 'UNEMPLOYED', 'MANUFACTURING', 'AGRICULTURE', 'MEDICAL','TRANSPORT', 'White', 'Black or African American', 'American Indian','Alaska Native', 'American Indian and Alaska Native tribes specified','Asian', 'Native Hawaiian', 'Some Other Race', 'Two or More Races', 'Private profit company employee','Local government employee', 'Private charitable company employee','Federal government employee', 'State government employee','Self-employed own business', 'Self-employed not own business','Unemployed', 'Non paying family business', 'Male', 'Female']]

#features = ['income', 'HISPANIC','EDUCATION', 'AGE','RETAIL','COMMUNITY_SERVICES', 'FINANCE', 'WHOLESALE','DOMESTIC_SERVICES', 'ENTERTAINMENT', 'ADMINISTRATION', 'UTILITIES','EXTRACTION', 'INFORMATION_TECH', 'CONSTRUCTION', 'SERVICES','MILITARY', 'UNEMPLOYED', 'MANUFACTURING', 'AGRICULTURE', 'MEDICAL','TRANSPORT', 'White', 'Black or African American', 'American Indian','Alaska Native', 'American Indian and Alaska Native tribes specified','Asian', 'Native Hawaiian', 'Some Other Race', 'Two or More Races', 'Private profit company employee','Local government employee', 'Private charitable company employee','Federal government employee', 'State government employee','Self-employed own business', 'Self-employed not own business','Unemployed', 'Non paying family business', 'Male', 'Female']
try:
    dataset = dataset[dataset.AGE != 'AGE']
except:
    pass
dataset.EDUCATION = dataset.EDUCATION.astype(float)
dataset.EDUCATION = stats.zscore(dataset.EDUCATION)

dataset.AGE = dataset.AGE.astype(int)
dataset.AGE = stats.zscore(dataset.AGE)

dataset.income = dataset.income.astype(float)
dataset.income = stats.zscore(dataset.income)

#dataset.to_csv("./flattened_files/flattened_with_years.csv", index=False, header = False, mode = 'a')
#dataset = dataset.drop(['DATA_FROM_YEAR'], axis = 1)
dataset.to_csv("./flattened_files/flattened_files_new/flattened_" + str(year) + "_with_year.csv", index=False, header = False)
year += 1

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
dataset.head(1000).to_csv("./flattened_2002_1000.csv", index=False, header=False)

In [87]:
dataset.to_csv("./flattened_2000_without_years.csv", index=False, header = False)

In [86]:
col = list(dataset.columns)
col.index('White')

22